In [153]:
#visualization part
#import matplotlib.pyplot as plt
#import seaborn as sns


#basic libs
import pandas as pd
import numpy as np
import random as rnd

from sklearn.model_selection import cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train_=pd.read_csv('../train_allcols.csv')
validate_=pd.read_csv('../validate_allcols.csv')
#test=pd.read_csv('../testwDSM.csv')

train_.shape, validate_.shape, #test.shape

((301619, 62), (148560, 62))

In [154]:
#train.describe()
train = train_.sample(10000)
validate = validate_.sample(10000)
train_.shape, #validate_.shape, validate.head(2)

((301619, 62),)

In [155]:
train = train.query('SUB2 != 1')
validate = validate.query('SUB2 != 1')

train = train[train.SUB2.isin([2,3,4,5,7,10])]
validate = validate[validate.SUB2.isin([2,3,4,5,7,10])]
print train.shape
print train['SUB2'].value_counts()

(4771, 62)
4     1690
2     1391
3      770
10     401
7      377
5      142
Name: SUB2, dtype: int64


In [156]:
# for sub2



X_train = train.drop(['SUB2'], axis=1)
Y_train = train["SUB2"]

from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif,chi2
#Selector_f = SelectPercentile(f_classif, percentile=25)
Selector_f = SelectKBest(f_classif, k=10)
Selector_f.fit(X_train,Y_train)

zipped = zip(X_train.columns.tolist(),Selector_f.scores_)
ans = sorted(zipped, key=lambda x: x[1])
for n,s in ans:    
    if 'FLG' in n: 
        pass
    else:
        print 'F-score: %3.2ft for feature %s' % (s,n)
                
#print X_train.columns.tolist()

F-score: 0.81t for feature PREG
F-score: 0.96t for feature DAYWAIT
F-score: 1.58t for feature VET
F-score: 1.81t for feature ARRESTS
F-score: 2.31t for feature CBSA10
F-score: 3.29t for feature DETNLF
F-score: 3.62t for feature PRIMPAY
F-score: 4.02t for feature ETHNIC
F-score: 4.69t for feature RACE
F-score: 4.83t for feature PRIMINC
F-score: 6.77t for feature LIVARAG
F-score: 6.80t for feature EMPLOY
F-score: 8.97t for feature PSYPROB
F-score: 9.31t for feature MARSTAT
F-score: 10.07t for feature SUB3
F-score: 10.27t for feature SERVSETA
F-score: 10.29t for feature STFIPS
F-score: 10.66t for feature REGION
F-score: 11.08t for feature ROUTE3
F-score: 12.66t for feature GENDER
F-score: 14.37t for feature HLTHINS
F-score: 15.62t for feature FRSTUSE1
F-score: 15.77t for feature YEAR
F-score: 15.86t for feature DETCRIM
F-score: 16.18t for feature CASEID
F-score: 17.38t for feature NOPRIOR
F-score: 18.59t for feature FREQ2
F-score: 19.11t for feature DIVISION
F-score: 19.15t for feature PS

In [157]:
# get the sorted feature list
import sys
for n, s in ans:
    
    if 'FLG' in n: 
        pass
    else:
        sys.stdout.write('\'%s\',' % (n))


'PREG','DAYWAIT','VET','ARRESTS','CBSA10','DETNLF','PRIMPAY','ETHNIC','RACE','PRIMINC','LIVARAG','EMPLOY','PSYPROB','MARSTAT','SUB3','SERVSETA','STFIPS','REGION','ROUTE3','GENDER','HLTHINS','FRSTUSE1','YEAR','DETCRIM','CASEID','NOPRIOR','FREQ2','DIVISION','PSOURCE','FREQ1','FRSTUSE3','EDUC','ROUTE1','METHUSE','NUMSUBS','SUB1','FREQ3','AGE','DSMCRIT','IDU','ROUTE2','ALCDRUG','FRSTUSE2',

In [158]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['SUB2',  'ROUTE2', 'FREQ2', 'FRSTUSE2', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'NUMSUBS'
             ]
retain_list = ['EMPLOY','GENDER','FREQ1','YEAR','EDUC','PSYPROB','PSOURCE','SERVSETA','DETCRIM',
               'REGION','NOPRIOR','DIVISION','DSMCRIT','ROUTE1','SUB1','AGE','IDU','SUB3','ROUTE3',
               'FREQ3','FRSTUSE3','FREQ2','FRSTUSE2']
X_train = train[retain_list]
Y_train = train["SUB2"]
X_validate = validate[retain_list]
Y_validate = validate["SUB2"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((4771, 23), (4784, 23))

In [159]:
print X_train.columns.tolist()

['EMPLOY', 'GENDER', 'FREQ1', 'YEAR', 'EDUC', 'PSYPROB', 'PSOURCE', 'SERVSETA', 'DETCRIM', 'REGION', 'NOPRIOR', 'DIVISION', 'DSMCRIT', 'ROUTE1', 'SUB1', 'AGE', 'IDU', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'FREQ2', 'FRSTUSE2']


In [160]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((4771, 184), (4784, 184))

In [161]:
# Logistic Regression
logreg = LogisticRegression(C=1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_validate, Y_validate) * 100, 2)
print acc_log


67.85


In [162]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
#Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_validate, Y_validate) * 100, 2)
print acc_perceptron


58.59


In [163]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


60.16


In [164]:
# Random Forest (slow)

random_forest = RandomForestClassifier(max_depth=20)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_validate, Y_validate) * 100, 2)
print acc_random_forest

#print cross_val_score(random_forest, X_validate, Y_validate)

61.68


In [165]:
from sklearn.ensemble import GradientBoostingClassifier,\
 AdaBoostClassifier, RandomForestClassifier
#Adaboost 
ada_boost = AdaBoostClassifier()
ada_boost.fit(X_train, Y_train) 

acc_ada = round(ada_boost.score(X_validate, Y_validate) * 100, 2)
print acc_ada


52.13


In [166]:
# Stochastic Gradient Descent
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


61.2


In [167]:
# Linear SVC
linear_svc = LinearSVC(C=1)
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_validate, Y_validate) * 100, 2)
print acc_linear_svc


67.66


In [168]:
print 'predict-top6-sub2-allcols-sample10000'
models = pd.DataFrame({
    'Model': [ 'Logistic Regression', 
              'Random Forest',  'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
               'AdaBoost'],
    'Validate Set Score': [acc_log, 
              acc_random_forest,  acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_ada]
    })
print models.sort_values(by='Validate Set Score', ascending=False)


predict-top6-sub2-allcols-sample10000
                        Model  Validate Set Score
0         Logistic Regression               67.85
4                  Linear SVC               67.66
1               Random Forest               61.68
3  Stochastic Gradient Decent               61.20
2                  Perceptron               58.59
5                    AdaBoost               52.13
